# Question 1

On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis.

### Average Analysis

In [1]:
import pandas as pd
import numpy as np

# Read CSV
df = pd.read_csv("sneakers.csv", index_col="order_id").dropna()

# Print variables
print("\nVariables:", list(df.columns))

# Conduct naive average order value (AOV) calculation
print("AOV:", "${:,.2f}".format(np.average(df["order_amount"])))

# Conduct more sensible average shoe value (ASV) calculation
print("ASV:", "${:,.2f}".format(np.average(df["order_amount"]/df["total_items"])))


Variables: ['shop_id', 'user_id', 'order_amount', 'total_items', 'payment_method', 'created_at']
AOV: $3,145.13
ASV: $387.74


We begin this section by reading in the .csv file that was provided. The data appears relatively clean, so we will simply drop all empty values and proceed with our analysis. Our second step is to explore our data by listing its variable names. Using these columns, we must calculate average order value and average shoe value.

### Outlier Analysis

In [2]:
from scipy import stats
from IPython.display import display

# Spacing
print()

# Calculate z-scores for "order_amount" and "total_items"
df["z-score"] = np.abs(stats.zscore(df[["order_amount", "total_items"]])).values.max(1)
df = df.sort_values("z-score", ascending=False)
display(df[df["z-score"] > 3].head(5))

# Remove obvious outlier
df = df[df["user_id"] != 607]

# Calculate new z-scores for "order_amount" and "total_items" after removing outlier
df["z-score"] = np.abs(stats.zscore(df[["order_amount", "total_items"]])).values.max(1)
df = df.sort_values("z-score", ascending=False)
display(df[df["z-score"] > 3].head(5))

# Find non-outlier observations
inliers = df[df["z-score"] < 3]

# Find how many shoes the typical customer orders
inlier_sizes = set(inliers["total_items"])
print("Typical order sizes:", inlier_sizes, "\n")

# Find the average order amount associated with typical customers
inlier_avgs = [np.average(inliers[inliers["total_items"] == x]["order_amount"]) for x in inlier_sizes]

# Cycle through each order size to check for any discernible relationship with order amount
for i, n in zip(inlier_sizes, inlier_avgs):
    print("Average AOV of order Size ", i, ": ", "${:,.2f}".format(n/i), sep="")

# Correct observations that appear to be outliers
df["order_amount"] = np.where(df["z-score"] > 3, 150 * df["total_items"], df["order_amount"])

,shop_id,user_id,order_amount,total_items,payment_method,created_at,z-score
order_id,,,,,,,
1363,42,607,704000,2000,credit_card,2017-03-15 4:00:00,17.120069
4057,42,607,704000,2000,credit_card,2017-03-28 4:00:00,17.120069
2970,42,607,704000,2000,credit_card,2017-03-28 4:00:00,17.120069
4647,42,607,704000,2000,credit_card,2017-03-02 4:00:00,17.120069
16,42,607,704000,2000,credit_card,2017-03-07 4:00:00,17.120069


,shop_id,user_id,order_amount,total_items,payment_method,created_at,z-score
order_id,,,,,,,
692,78,878,154350,6,debit,2017-03-27 22:51:43,28.906408
2493,78,834,102900,4,debit,2017-03-04 4:37:34,19.223633
1260,78,775,77175,3,credit_card,2017-03-27 9:27:20,14.382245
2907,78,817,77175,3,debit,2017-03-16 3:45:46,14.382245
4193,78,787,77175,3,credit_card,2017-03-18 9:25:32,14.382245


Typical order sizes: {1, 2, 3, 4} 

Average AOV of order Size 1: $151.85
Average AOV of order Size 2: $151.76
Average AOV of order Size 3: $152.44
Average AOV of order Size 4: $149.63


In this section, we must remove outliers to find more useful estimate(s) of value. To do this, we calculate the z-scores of `order_amount` and `total_items`. We then store the maximum of these two z-scores because we do not need to distinguish between which variable is unusual. Next, we remove user 607 because their orders are both repetitive and unusual. We then conduct this analysis a second time. We find the AOV associated with a typical customer. Finally, we apply this average to our outliers and adjust for order size.

### New Average Analysis

In [3]:
# Conduct new AOV calculation
print("\nNew AOV:", "${:,.2f}".format(np.average(df["order_amount"])))

# Conduct new ASV calculation
print("New ASV:", "${:,.2f}".format(np.average(df["order_amount"]/df["total_items"])))


New AOV: $302.34
New ASV: $151.75


Here, we must recalculate average order value and average shoe value.

#### A) What could be going wrong with our calculation?
There are two issues that are leading to the perceived "error". Firstly, what was calculated was average order value while what we are interested in is average shoe value. That is, the average order consists of multiple shoes. Secondly, we failed to preclude the possibility of outliers before conducting our analysis.

#### B) What metric should we report for this dataset?
We should report average shoe value alongside average order value with an emphasis on the meaning of both.

#### C) What is its value?
The average <i>order</i> value is <span>$</span>302.34. The average <i>shoe</i> value is <span>$</span>151.75

# Question 2

For this question we’ll need to use SQL queries.

### First Query

<code style="width: 100%;">SELECT COUNT(*) FROM Orders <span style="color: #008000">-- Select number of orders</span>
 JOIN Shippers ON Shippers.ShipperID = Orders.ShipperID <span style="color: #008000">-- Retrieve shipping information</span>
 WHERE ShipperName = "Speedy Express" <span style="color: #008000">-- Filter out other shippers</span>
<br>Number of Records: 1
 COUNT(*)
 54
</code>

### Second Query

<code style="width: 100%;">SELECT LastName, COUNT(*) FROM <span style="color: #008000">-- Select number of orders for each LastName</span>
 (
  SELECT * FROM Orders
  JOIN Employees ON Employees.EmployeeID = Orders.EmployeeID <span style="color: #008000">-- Retrieve employee information</span>
 )
 GROUP BY LastName
 ORDER BY COUNT(*) DESC <span style="color: #008000">-- Put the best employee at top</span>
 LIMIT 1 <span style="color: #008000">-- Select only the best employee</span>
<br>Number of Records: 1
 LastName       COUNT(*)
 Peacock        40
</code>

### Third Query

<code style="display: block; width: 100%;">SELECT ProductName, COUNT(*) FROM <span style="color: #008000">-- Select number of orders for each product</span>
 (
  SELECT * FROM Orders <span style="color: #008000">-- Select Orders</span>
  JOIN Customers ON Customers.CustomerID = Orders.CustomerID <span style="color: #008000">-- Retrieve customer information</span>
  JOIN OrderDetails ON OrderDetails.OrderID = Orders.OrderID <span style="color: #008000">-- Retrieve order details</span>
  JOIN Products ON Products.ProductID = OrderDetails.ProductID <span style="color: #008000">-- Retrieve product information</span>
  WHERE Country = "Germany" <span style="color: #008000">-- Filter out non-German customers</span>
 )
 GROUP BY ProductName
 ORDER BY COUNT(*) DESC <span style="color: #008000">-- Put the best product at top</span>
 LIMIT 1 <span style="color: #008000">-- Select only the best product</span>
<br>Number of Records: 1
 ProductName            COUNT(*)
 Gorgonzola Telino      5
</code>

#### A) How many orders were shipped by Speedy Express in total?
According to our first query, Speedy Express shipped 54 orders.
>54

#### B) What is the last name of the employee with the most orders?
According to our second query, Mrs. Peacock had 40 orders.
>Peacock

#### C) What product was ordered the most by customers in Germany?
According to our third query, Gorgonzola Telino was the product most ordered by Germans.
>Gorgonzola Telino